In [ ]:
import pandas as pd
import os
import numpy as np
import tqdm
os.chdir(r'D:\Projects\football-odds-analysis')
from utils.preprocess import preprocess_odds_results, preprocess_pipeline
from input_variables import COMPANY_BETS, CO_CODES
from utils.bet_payout_engine import get_bet_results

N_CUTS_PROBABILITIES = 10
intervals = np.linspace(0, 1, N_CUTS_PROBABILITIES+1)
deciles_returns = {}

for d in range(N_CUTS_PROBABILITIES):
    deciles_returns[d] = []

bookmakers_data = preprocess_pipeline('all_avail_games.csv')
odds_results_data = preprocess_odds_results('all_avail_games.csv')
bookmakers_odds = odds_results_data['Odds']
bookmakers_deciles = pd.DataFrame()
game_results = odds_results_data['Results']

for c in COMPANY_BETS:
    bookmakers_deciles[c] = pd.cut((1/bookmakers_odds[c]), intervals, labels=False)

for game in tqdm.tqdm(bookmakers_odds.index):
    for c in COMPANY_BETS:
        decile = bookmakers_deciles.loc[game, c]
        bet = c[-1]
        company = CO_CODES[c[:-1]]
        deciles_returns[decile].append(get_bet_results(bookmakers_data, game, bet, company, 1) - 1)